## Rigid Body Dynamics

In [32]:
import numpy as np
import sympy as sp
from sympy import Matrix, diff
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [33]:
# Helper function to compute the cross-product matrix [v]_x
def cross_matrix(v):
    return np.array([[0, -v[2], v[1]],
                     [v[2], 0, -v[0]],
                     [-v[1], v[0], 0]])

In [34]:
# Helper function to compute the vector from a cross-product matrix
def uncross_matrix(m):
    return np.array([m[2, 1], m[0, 2], m[1, 0]])

In [35]:
# Dynamics function to compute the time derivative of angular velocity
def omega_dot(t, omega, I_B):
    cross_mat = cross_matrix(omega)
    omega_dot = -np.linalg.inv(I_B) @ (cross_mat @ I_B @ omega)
    return omega_dot

In [36]:
# Moment of inertia matrix
I_B = np.array([[10, 0, 0],
               [0, 5, 0],
               [0, 0, 1]])

# Initial conditions
w0_1 = np.array([0.9, 0.1, 0])
w0_2 = np.array([0.1, 0.9, 0])

# Time
dt = 0.001
t = np.linspace(0,10,int(10/dt))

In [37]:
# Numerical integration of EoM solve for w
sol_1 = solve_ivp(lambda t, y: omega_dot(t, y, I_B),
                             [0, 10], w0_1, 
                             t_eval=t,
                             rtol=1e-8, atol=1e-8)
sol_2 = solve_ivp(lambda t, y: omega_dot(t, y, I_B),
                             [0, 10], w0_2, 
                             t_eval=t,
                             rtol=1e-8, atol=1e-8)
sol_1

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  1.000e-03 ...  9.999e+00  1.000e+01]
        y: [[ 9.000e-01  9.000e-01 ...  9.011e-01  9.011e-01]
            [ 1.000e-01  1.000e-01 ... -3.074e-02 -3.100e-02]
            [ 0.000e+00  4.500e-04 ...  1.586e-01  1.585e-01]]
      sol: None
 t_events: None
 y_events: None
     nfev: 1028
     njev: 0
      nlu: 0

In [38]:
# Plotting components of angular velocity versus time
plt.figure()
plt.subplot(2, 1, 1)
plt.plot(sol_1.t, sol_1.y[0, :], label='components 1')
plt.plot(sol_1.t, sol_1.y[1, :], label='components 2')
plt.plot(sol_1.t, sol_1.y[2, :], label='components 3')
plt.title('Angular Velocity Components - Case 1')
plt.xlabel('Time (s)')
plt.ylabel('Angular Velocity')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(sol_2.t, sol_2.y[0, :], label='components 1')
plt.plot(sol_2.t, sol_2.y[1, :], label='components 2')
plt.plot(sol_2.t, sol_2.y[2, :], label='components 3')
plt.title('Angular Velocity Components - Case 2')
plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Angular Velocity')

plt.tight_layout()
plt.show()

In [39]:
# Calculate angular kinetic energy over time
energy_1 = np.sum((sol_1.y.T @ I_B @ sol_1.y) / 2, axis=0)
energy_2 = np.sum((sol_2.y.T @ I_B @ sol_2.y) / 2, axis=0)

In [40]:
# Plotting angular kinetic energy versus time
plt.figure(figsize=(8, 5))
plt.plot(sol_1.t, energy_1, label='Case 1')
plt.plot(sol_2.t, energy_2, label='Case 2')
plt.title('Angular Kinetic Energy vs Time')
plt.xlabel('Time (s)')
plt.ylabel('Angular Kinetic Energy')
plt.legend()
plt.show()

## Problem 3

In [41]:
# Define symbols
theta3, theta2, theta1, t = sp.symbols('theta3 theta2 theta1 t', real=True)
omega_x, omega_y, omega_z = sp.symbols('omega_x omega_y omega_z', real=True)

# Make rotation angles functions of time so we can take derivatives
theta3 = sp.Function('theta3', real=True)(t)
theta2 = sp.Function('theta2', real=True)(t)
theta1 = sp.Function('theta1', real=True)(t)

In [72]:
# Define the rotation matrices
Rz_theta3 = sp.Matrix([
    [sp.cos(theta3), -sp.sin(theta3), 0],
    [sp.sin(theta3), sp.cos(theta3), 0],
    [0, 0, 1]])

Ry_theta2 = sp.Matrix([
    [sp.cos(theta2), 0, sp.sin(theta2)],
    [0, 1, 0],
    [-sp.sin(theta2), 0, sp.cos(theta2)]])

Rx_theta1 = sp.Matrix([
    [1, 0, 0],
    [0, sp.cos(theta1), -sp.sin(theta1)],
    [0, sp.sin(theta1), sp.cos(theta1)]])

# Calculate the overall rotation matrix R
R = sp.simplify(Rz_theta3 @ Ry_theta2 @ Rx_theta1)
R

Matrix([
[cos(theta2(t))*cos(theta3(t)), sin(theta1(t))*sin(theta2(t))*cos(theta3(t)) - sin(theta3(t))*cos(theta1(t)),  sin(theta1(t))*sin(theta3(t)) + sin(theta2(t))*cos(theta1(t))*cos(theta3(t))],
[sin(theta3(t))*cos(theta2(t)), sin(theta1(t))*sin(theta2(t))*sin(theta3(t)) + cos(theta1(t))*cos(theta3(t)), -sin(theta1(t))*cos(theta3(t)) + sin(theta2(t))*sin(theta3(t))*cos(theta1(t))],
[              -sin(theta2(t)),                                                sin(theta1(t))*cos(theta2(t)),                                                 cos(theta1(t))*cos(theta2(t))]])

In [ ]:
theta = sp.Matrix([[theta1],[theta2],[theta3]])
theta_dot = sp.diff(theta,t)
theta_dot

In [80]:
R_dot = sp.diff(R,t)
omega = sp.simplify(R.T @ R_dot)
omega_x = omega[2,1]
omega_y = omega[0,2]
omega_z = -omega[0,1]

w11 = sp.diff(omega_x, sp.diff(theta1, t))
w12 = sp.diff(omega_x, sp.diff(theta2, t))
w13 = sp.diff(omega_x, sp.diff(theta3, t))

w21 = sp.diff(omega_y, sp.diff(theta1, t))
w22 = sp.diff(omega_y, sp.diff(theta2, t))
w23 = sp.diff(omega_y, sp.diff(theta3, t))

w31 = sp.diff(omega_z, sp.diff(theta1, t))
w32 = sp.diff(omega_z, sp.diff(theta2, t))
w33 = sp.diff(omega_z, sp.diff(theta3, t))

W = sp.Matrix([
    [w11, w12, w13],
    [w21, w22, w23],
    [w31, w32, w33]])

W_new = sp.Matrix.inv(W)
W_new = sp.simplify(W_new)
W_new

Matrix([
[1, sin(theta1(t))*tan(theta2(t)), cos(theta1(t))*tan(theta2(t))],
[0,                cos(theta1(t)),               -sin(theta1(t))],
[0, sin(theta1(t))/cos(theta2(t)), cos(theta1(t))/cos(theta2(t))]])